In [46]:
import pandas as pd
import sqlalchemy

# Movie Ratings

You've started a new movie-rating website, and you've been collecting data on reviewers' ratings of various movies. There's not much data yet, but you can still try out some interesting queries. Here's the schema: 

Movie ( mID, title, year, director ) 
English: There is a movie with ID number mID, a title, a release year, and a director. 

Reviewer ( rID, name ) 
English: The reviewer with ID number rID has a certain name. 

Rating ( rID, mID, stars, ratingDate ) 
English: The reviewer rID gave the movie mID a number of stars rating (1-5) on a certain ratingDate. 

## Connect to the database

In [3]:
# Connect to the movie db
engine = sqlalchemy.create_engine(
    "postgresql+psycopg2://", 
    connect_args={"database": "postgres", "user": "sherlock", "host": "/var/run/postgresql"}
)
con = engine.connect()
con.execute("SET schema 'input'")

## Load the data in the db
If you want to reset the database, simply rerun this cell

In [4]:
from sqlalchemy.sql import text
con.execute(open("movie-ratings.sql").read())
con.execute(open("social.sql").read())

In [4]:
q = """
SELECT * 
FROM Rating 
limit 3
"""
df = pd.read_sql(q, con)
df

,rid,mid,stars,ratingdate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None


Find the titles of all movies directed by Steven Spielberg. 


In [12]:
qt = """
SELECT title
FROM Movie
WHERE director = 'Steven Spielberg'
"""
dft = pd.read_sql(qt,con)
dft

,title
0,E.T.
1,Raiders of the Lost Ark


Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order. 

In [18]:
qr = """
SELECT year
FROM Movie, Rating 
WHERE Movie.mID = Rating.mID and stars >=4 
ORDER BY year ASC

"""
dfr = pd.read_sql(qr,con)
dfr

,year
0,1937
1,1937
2,1939
3,1981
4,1981
5,2009


Find the titles of all movies that have no ratings. 


In [22]:
q = """
SELECT title
FROM Movie
WHERE mID NOT in (SELECT mID from Rating)
"""
df = pd.read_sql(q,con)
df

,title
0,Star Wars
1,Titanic


Some reviewers didn't provide a date with their rating. Find the names of all reviewers who have ratings with a NULL value for the date. 


In [29]:
q = """
SELECT name 
FROM Reviewer, Rating
WHERE Reviewer.rID = Rating.rID and ratingdate is NULL
"""
df = pd.read_sql(q,con)
df

,name
0,Daniel Lewis
1,Chris Jackson


Write a query to return the ratings data in a more readable format: reviewer name, movie title, stars, and ratingDate. Also, sort the data, first by reviewer name, then by movie title, and lastly by number of stars. 
 

In [31]:
q = """
SELECT name, title, stars, ratingdate
FROM Reviewer, Movie, Rating
WHERE Reviewer.rID = Rating.rID and Movie.mID = Rating.mID
ORDER BY name, title, stars;
"""
df = pd.read_sql(q,con)
df

,name,title,stars,ratingdate
0,Ashley White,E.T.,3,2011-01-02
1,Brittany Harris,Raiders of the Lost Ark,2,2011-01-30
2,Brittany Harris,Raiders of the Lost Ark,4,2011-01-12
3,Brittany Harris,The Sound of Music,2,2011-01-20
4,Chris Jackson,E.T.,2,2011-01-22
5,Chris Jackson,Raiders of the Lost Ark,4,None
6,Chris Jackson,The Sound of Music,3,2011-01-27
7,Daniel Lewis,Snow White,4,None
8,Elizabeth Thomas,Avatar,3,2011-01-15
9,Elizabeth Thomas,Snow White,5,2011-01-19


For all cases where the same reviewer rated the same movie twice and gave it a higher rating the second time, return the reviewer's name and the title of the movie. 

In [33]:
q = """
SELECT name, title
FROM Reviewer, Movie, (SELECT R1.rID, R1.mID 
    FROM Rating R1, Rating R2
    WHERE R1.rID = R2.rID and R1.mID = R2.mID 
        and R2.stars > R1.stars and R2.ratingdate > R1.ratingdate)C
WHERE C.mid = Movie.mid and C.rid = Reviewer.rid
"""
df = pd.read_sql(q,con)
df

,name,title
0,Sarah Martinez,Gone with the Wind


For each movie that has at least one rating, find the highest number of stars that movie received. Return the movie title and number of stars. Sort by movie title. 

In [45]:
q = """
SELECT title, stars
FROM Movie, (SELECT Movie.mID, stars
              from Movie, Rating
              where Movie.mID = Rating.mID
    except    
    SELECT R1.mID, R1.stars 
    FROM Rating R1, Rating R2
    WHERE R1.mID = R2.mID and R1.stars < R2.stars) HSTAR
WHERE Movie.mID = HSTAR.mID
ORDER BY title;
"""
df = pd.read_sql(q,con)
df

,title,stars
0,Avatar,5
1,E.T.,3
2,Gone with the Wind,4
3,Raiders of the Lost Ark,4
4,Snow White,5
5,The Sound of Music,3


For each movie, return the title and the 'rating spread', that is, the difference between highest and lowest ratings given to that movie. Sort by rating spread from highest to lowest, then by movie title. 

In [48]:
q="""
SELECT title, spread
FROM Movie, (
    SELECT mID, max(stars) - min(stars) AS spread
    FROM Rating
    GROUP BY mID
    ) RatingSpread
WHERE Movie.mID = RatingSpread.mID
ORDER BY spread DESC, title
"""
df = pd.read_sql(q,con)
df

,title,spread
0,Avatar,2
1,Gone with the Wind,2
2,Raiders of the Lost Ark,2
3,E.T.,1
4,Snow White,1
5,The Sound of Music,1


Find the difference between the average rating of movies released before 1980 and the average rating of movies released after 1980. (Make sure to calculate the average rating for each movie, then the average of those averages for movies before 1980 and movies after. Don't just calculate the overall average rating before and after 1980.) 

In [53]:
q = """
select avg(before_80.group_avg) - avg(post_80.group_avg) as difference
from (
  select Rating.mID, avg(stars) as group_avg
  from Rating, Movie
  where Rating.mID = Movie.mID
  and year <= 1980
  group by Rating.mID
) as before_80,
(
  select Rating.mID, avg(stars) as group_avg
  from Rating, Movie
  where Rating.mID = Movie.mID
  and year > 1980
  group by Rating.mID
) as post_80
"""
df = pd.read_sql(q,con)
df

,difference
0,0.055556


# The Social Network

Students at your hometown high school have decided to organize their social network using databases. So far, they have collected information about sixteen students in four grades, 9-12. Here's the schema: 

Highschooler ( ID, name, grade ) 
English: There is a high school student with unique ID and a given first name in a certain grade. 

Friend ( ID1, ID2 ) 
English: The student with ID1 is friends with the student with ID2. Friendship is mutual, so if (123, 456) is in the Friend table, so is (456, 123). 

Likes ( ID1, ID2 ) 
English: The student with ID1 likes the student with ID2. Liking someone is not necessarily mutual, so if (123, 456) is in the Likes table, there is no guarantee that (456, 123) is also present. 

<img src="social.png">

In [ ]:
df = pd.read_sql("SELECT * from Highschooler limit 3", con)
df

Find the names of all students who are friends with someone named Gabriel. 


For every student who likes someone 2 or more grades younger than themselves, return that student's name and grade, and the name and grade of the student they like. 

For every pair of students who both like each other, return the name and grade of both students. Include each pair only once, with the two names in alphabetical order. 

Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade. 

For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades. 

Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name within each grade. 

For each student A who likes a student B where the two are not friends, find if they have a friend C in common (who can introduce them!). For all such trios, return the name and grade of A, B, and C. 

Find the difference between the number of students in the school and the number of different first names. 


Find the name and grade of all students who are liked by more than one other student. 


## Modification

Add the reviewer Roger Ebert to your database, with an rID of 209. 


In [ ]:
q = "insert into Reviewer(rID, name) values (209, 'Roger Ebert')"
con.execute(q)

Insert 5-star ratings by James Cameron for all movies in the database. Leave the review date as NULL. 


For all movies that have an average rating of 4 stars or higher, add 25 to the release year. (Update the existing tuples; don't insert new tuples.) 

Remove all ratings where the movie's year is before 1970 or after 2000, and the rating is fewer than 4 stars. 

It's time for the seniors to graduate. Remove all 12th graders from Highschooler. 


If two students A and B are friends, and A likes B but not vice-versa, remove the Likes tuple. 


For all cases where A is friends with B, and B is friends with C, add a new friendship for the pair A and C. Do not add duplicate friendships, friendships that already exist, or friendships with oneself. (This one is a bit challenging; congratulations if you get it right.) 